# Подготовка данных

In [1]:
import pandas as pd

train_data_fast_text = pd.read_parquet('data_fusion_train.parquet', engine='pyarrow')

In [2]:
train_data_fast_text = train_data_fast_text[train_data_fast_text.category_id == -1].drop_duplicates('item_name')


In [10]:
from tqdm import tqdm
import re

product_names_arr = [[s for s in re.sub("[^а-я$]", " ", i.lower()).split(' ') if len(s) > 1] for i in tqdm(train_data_fast_text["item_name"].tolist())]


100%|██████████| 3107502/3107502 [00:37<00:00, 83044.81it/s] 


# Модель

In [13]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath


model = FastText(size=100)

# build the vocabulary
model.build_vocab(sentences=product_names_arr)



In [15]:
# train the model
model.train(
    sentences=product_names_arr, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

In [16]:
print(model)


## Сохраняем модель

In [17]:
import tempfile
import os


with tempfile.NamedTemporaryFile(prefix='saved_model_gensim-', delete=False) as tmp:
    model.save(tmp.name, separately=[])

# Load back the same model.
loaded_model = FastText.load(tmp.name)
print(loaded_model)

os.unlink(tmp.name)  # demonstration complete, don't need the temp file anymore


In [30]:
model.wv